In [1]:
from datasets import load_dataset
train_dataset = load_dataset('imagefolder',data_dir='./Landmark_Retrieval/Landmark_Retrieval/train/',split='train',drop_labels=False)

c:\Users\Asus\anaconda3\envs\info_retrieve\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 1500/1500 [00:00<00:00, 29670.10it/s]
Found cached dataset imagefolder (C:/Users/Asus/.cache/huggingface/datasets/imagefolder/default-8f2e112e271d2f44/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [2]:
train_dataset[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1500x924>,
 'label': 0}

In [3]:
label2id = {label:idx for idx,label in enumerate(train_dataset.features['label'].names)}
id2label = {idx:label for idx,label in enumerate(train_dataset.features['label'].names)}

In [4]:
label2id

{'bao_tang_ha_noi': 0,
 'buu_dien_trung_tam_tphcm': 1,
 'cau_long_bien': 2,
 'cau_nhat_tan': 3,
 'cau_rong': 4,
 'cho_ben_thanh_tphcm': 5,
 'chua_cau': 6,
 'chua_mot_cot': 7,
 'chua_thien_mu': 8,
 'cot_co': 9,
 'hoang_thanh': 10,
 'hon_chong_nhatrang': 11,
 'landmark81': 12,
 'lang_bac': 13,
 'lang_khai_dinh': 14,
 'mui_ca_mau': 15,
 'mui_ke_ga_phanthiet': 16,
 'nha_hat_lon_hanoi': 17,
 'nha_hat_lon_tphcm': 18,
 'nha_tho_da_co_sapa': 19,
 'nha_tho_lon_ha_noi': 20,
 'quang_truong_lam_vien': 21,
 'suoi_tien_tphcm': 22,
 'thac_ban_gioc': 23,
 'thap_cham': 24,
 'thap_rua': 25,
 'toa_nha_bitexco_tphcm': 26,
 'tuong_chua_kito_vungtau': 27,
 'ubnd_tphcm': 28,
 'van_mieu': 29}

In [5]:
from transformers import AutoImageProcessor
checkpoint = './results/checkpoint-500'
# processor = AutoImageProcessor.from_pretrained(checkpoint)

In [6]:
import evaluate
import numpy as numpy
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = numpy.argmax(logits,axis=1)
    return accuracy.compute(predictions,labels)

In [7]:
from transformers import AutoFeatureExtractor, AutoModel
# extractor = AutoFeatureExtractor.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to('cuda')

Some weights of the model checkpoint at ./results/checkpoint-500 were not used when initializing MobileNetV2Model: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing MobileNetV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileNetV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
import torch
from torchvision.transforms import ToTensor, Compose,CenterCrop,Normalize
pipe = Compose([CenterCrop(224),ToTensor(),Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])
def preprocess_function(examples):
    examples['pixel_values']= [pipe(image.convert('RGB')) for image in examples['image']]
    return examples

In [9]:
train_dataset = train_dataset.map(preprocess_function, batched=True,batch_size=32)
train_dataset.set_format(type='torch', columns=['pixel_values', 'label'])

Loading cached processed dataset at C:\Users\Asus\.cache\huggingface\datasets\imagefolder\default-8f2e112e271d2f44\0.0.0\37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-23b2b45667b70ba1.arrow


In [10]:
import torch
def get_embeddings(examples):
    # image = torch.stack(examples['pixel_values']).to('cuda')
    with torch.no_grad():
        image = examples['pixel_values'].to('cuda')
        outputs = model(image)
        embeddings = outputs.pooler_output
    return {'embeddings':embeddings}
    


In [11]:
temp_dataset = train_dataset.select(range(2))
temp_dataset = temp_dataset.map(get_embeddings,batched=True,batch_size=1)
temp_dataset

Loading cached processed dataset at C:\Users\Asus\.cache\huggingface\datasets\imagefolder\default-8f2e112e271d2f44\0.0.0\37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-e06db96cb24f6e5a.arrow


Dataset({
    features: ['image', 'label', 'pixel_values', 'embeddings'],
    num_rows: 2
})

In [12]:
# another = train_dataset.select(range(15))
# temp_out= model(another['pixel_values'].to('cuda')).pooler_output
# temp = {}
# temp['embeddings'] = temp_out

In [13]:
# model(temp_dataset['pixel_values'].to('cuda'))

In [14]:
# temp_dataset = train_dataset.select(range(1))
# temp_dataset = temp_dataset.map(get_embeddings,batched=True,remove_columns=['image','label'])

In [15]:
candidate_embeddings = train_dataset.map(get_embeddings,batched=True,batch_size=32)

Loading cached processed dataset at C:\Users\Asus\.cache\huggingface\datasets\imagefolder\default-8f2e112e271d2f44\0.0.0\37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-114ddb7d55286322.arrow


In [16]:
from tqdm import tqdm
candidate_ids = []
for id in tqdm(range(len(candidate_embeddings))):
    label = candidate_embeddings[id]['label'].item()
    entry = str(id) + '-' + str(id2label[label])
    candidate_ids.append(entry)

100%|██████████| 1500/1500 [00:15<00:00, 97.49it/s] 


In [17]:
import numpy as np
all_embeddings = np.array(candidate_embeddings['embeddings'])
all_embeddings = torch.from_numpy(all_embeddings).to('cuda')

In [18]:
def compute_sim(emb_one,emb_two):
    return torch.cosine_similarity(emb_one,emb_two,dim=1).cpu().numpy().tolist()

def fetch_sim(image,top_k=10):
    # image = pipe(image.convert('RGB')).unsqueeze(0).to('cuda')
    # img_pipe = Compose([CenterCrop(224),Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])
    image = image.unsqueeze(0).to('cuda')
    new_batch = {'pixel_values':image}
    with torch.no_grad():
        new_batch = model(**new_batch).pooler_output
    sim = compute_sim(new_batch,all_embeddings)
    similarity_mapping = dict(zip(candidate_ids,sim))
    sorted_similarity = dict(sorted(similarity_mapping.items(),key=lambda x:x[1],reverse=True))
    id_entries = list(sorted_similarity.keys())[:top_k]
    ids = list(map(lambda x:x.split('-')[0],id_entries))
    labels = list(map(lambda x:x.split('-')[1],id_entries))
    return ids,labels

In [19]:
test_dataset = load_dataset('imagefolder',data_dir='./Landmark_Retrieval/Landmark_Retrieval/test/',split='train',drop_labels=False)
test_dataset = test_dataset.map(preprocess_function, batched=True,batch_size=32)
test_dataset.set_format(type='torch', columns=['pixel_values', 'label'])

Resolving data files: 100%|██████████| 1500/1500 [00:00<00:00, 34065.32it/s]
Found cached dataset imagefolder (C:/Users/Asus/.cache/huggingface/datasets/imagefolder/default-19797c74a360d3b4/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Loading cached processed dataset at C:\Users\Asus\.cache\huggingface\datasets\imagefolder\default-19797c74a360d3b4\0.0.0\37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-94e8d5d89d4a6576.arrow


In [26]:
import random
# random.randint(0,len(test_dataset)
image_num = random.randint(0,len(train_dataset))
print(image_num)
print(id2label[train_dataset[image_num].get('label').item()])
test_sample = train_dataset[image_num]['pixel_values']
# print(test_sample)
sim = fetch_sim(test_sample)
for i in zip(sim[0],sim[1]):
    print(i)

1155
thac_ban_gioc
('1155', 'thac_ban_gioc')
('1198', 'thac_ban_gioc')
('1150', 'thac_ban_gioc')
('1162', 'thac_ban_gioc')
('1164', 'thac_ban_gioc')
('1171', 'thac_ban_gioc')
('1187', 'thac_ban_gioc')
('1195', 'thac_ban_gioc')
('1193', 'thac_ban_gioc')
('1190', 'thac_ban_gioc')


In [25]:
len(train_dataset)

1500

In [21]:
test_dataset[0]['pixel_values']

tensor([[[ 1.3242,  1.3242,  1.3242,  ...,  1.9235,  1.9578,  1.9749],
         [ 1.3413,  1.3242,  1.2728,  ...,  1.9407,  1.9578,  1.9578],
         [ 1.3070,  1.2385,  1.2557,  ...,  1.9235,  1.9407,  1.9578],
         ...,
         [-1.3302, -1.4329, -1.4672,  ..., -1.1932, -1.3302, -1.4500],
         [-0.7993, -0.8507, -0.7308,  ..., -1.2445, -1.5699, -1.6384],
         [-0.8849, -0.7650, -0.7137,  ..., -1.4158, -1.7240, -1.7069]],

        [[ 1.8158,  1.8333,  1.8508,  ...,  2.2710,  2.2885,  2.3060],
         [ 1.8508,  1.8508,  1.8508,  ...,  2.2710,  2.2885,  2.2885],
         [ 1.9559,  1.8859,  1.9034,  ...,  2.2710,  2.2710,  2.2885],
         ...,
         [-0.6176, -0.7752, -0.9153,  ..., -0.4601, -0.6001, -0.7402],
         [-0.1800, -0.2675, -0.2500,  ..., -0.5301, -0.8452, -0.9328],
         [-0.3725, -0.2675, -0.2850,  ..., -0.7577, -1.0728, -1.0203]],

        [[ 2.6400,  2.6400,  2.6051,  ...,  2.6400,  2.6400,  2.6400],
         [ 2.6400,  2.5703,  2.4831,  ...,  2

In [22]:
test_dataset[0].keys()

dict_keys(['label', 'pixel_values'])

In [23]:
from PIL import Image
image = Image.fromarray(test_dataset[0]['pixel_values'].to('cpu').numpy())

TypeError: Cannot handle this data type: (1, 1, 224), <f4